In [1]:
import utils
import pyteomics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
%matplotlib inline

In [2]:
from pyteomics import mzxml,auxiliary
with mzxml.read("./data010.mzXML") as reader:
    polarity = [spectrum['polarity'] for spectrum in reader]
with mzxml.read("./data010.mzXML") as reader:
    spectrum1 = next(reader)

In [3]:
ppm = 1e-6

In [4]:
def avg_ppm(spectrum,ppm_num):
    #平均质谱图
    spectrum.sort_values(by='mz',inplace=True)
    spectrum['grp'] = (spectrum.mz.diff()>spectrum.mz*ppm_num).cumsum()
    grp = spectrum.groupby('grp')
    mz = grp.apply(lambda x: x.mz.dot(x.intensity/x.intensity.sum()))
    mz.name = 'mz'
    intensity = grp['intensity'].mean()
    avg = pd.concat([mz,intensity],axis=1)
    avg.index.name = None
    avg = avg.loc[:,['mz','intensity']]
    avg['charge']=1
    return avg

In [5]:
def isotope_mv(avgmass,isotype):
    #选取同位素峰
    avgmass.sort_values(by='mz',inplace = True)
    avgmass['grp'] = (avgmass.mz.diff()-isotype>.01).cumsum()
    grp = avgmass.groupby('grp')
    charge_mask = grp['mz'].count()>1
    isotopes = avgmass.loc[grp['intensity'].idxmax()]#强度最大的同位素峰组, 不是iloc
    isotopes = isotopes.copy()
    isotopes.loc[charge_mask.values,'charge']=1/isotype
    isotopes = isotopes.loc[:,['mz','intensity','charge']]
    return isotopes

In [6]:
def f1(s1,theshold=1e-3):
    return isotope_mv(isotope_mv(avg_ppm(s1,theshold),.5),1)

In [7]:
def cov1(spectrum1):
    s1 = pd.DataFrame(spectrum1).loc[:,['retentionTime','m/z array','intensity array']]
    s1.columns = ['rt','mz','intensity']
    return f1(s1)

avgmass = a
isotype = 1
avgmass.sort_values(by='mz',inplace = True)
avgmass['grp'] = (avgmass.mz.diff()-isotype>.01).cumsum()
grp = avgmass.groupby('grp')
charge_mask = grp['mz'].count()>1
isotopes = avgmass.loc[grp['intensity'].idxmax()]#强度最大的同位素峰组
isotopes = isotopes.copy()
isotopes.loc[charge_mask.values,'charge']=1/isotype
isotopes = isotopes.loc[:,['mz','intensity','charge']]

# 去除噪音
# 同位素 .5, 1

In [ ]:
with mzxml.read("./data010.mzXML") as reader:
    ss = [cov1(spectrum) for spectrum in reader]